In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
def generate_time_series(batch_size,n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4,batch_size,1)
    time = np.linspace(0,1,n_steps)
    series = 0.5* np.sin(time-offsets1)*(freq1*10+10)
    series += 0.2* np.sin(time-offsets2)*(freq2*20+20)
    series += 0.1* (np.random.rand(batch_size,n_steps)-0.5)
    return series[...,np.newaxis].astype(np.float32)
    

In [4]:
n_steps = 50
series = generate_time_series(10000,n_steps+1)
x_train, y_train = series[:7000,:n_steps], series[:7000,-1]
x_valid, y_valid = series[7000:9000,:n_steps], series[7000:9000,-1]
x_test, y_test = series[9000:,:n_steps],series[9000:,-1]

In [5]:
y_pred = x_valid[:,-1]
np.mean(keras.losses.mean_squared_error(y_valid,y_pred))

0.055072278

In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50,1]),
    keras.layers.Dense(1)
])

In [7]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1,input_shape=[None,1])
])

In [8]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
    keras.layers.SimpleRNN(20,return_sequences=True),
    keras.layers.SimpleRNN(1)
])

In [9]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

In [10]:
series = generate_time_series(1,n_steps+10)
x_new, y_new = series[:,:n_steps],series[:,n_steps:]
x = x_new
for step_ahead in range(10):
    y_pred_one= model.predict(x[:,step_ahead:])[:,np.newaxis,:]
    x = np.concatenate([x,y_pred_one],axis=1)
y_pred = x[:,n_steps:]

In [11]:
series = generate_time_series(10000,n_steps+10)
x_train, y_train = series[:7000,:n_steps], series[:7000,-10:,0]
x_valid, y_valid = series[7000:9000,:n_steps], series[7000:9000,-10:,0]
x_test, y_test = series[9000:,:n_steps], series[9000:,-10:,0]

In [12]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

In [13]:
y_pred = model.predict(x_new)
y_pred

array([[-0.77066565, -0.25399882,  1.6139213 ,  1.5800487 , -0.9297331 ,
         0.86407363,  0.05557621, -0.8509676 ,  1.1286358 ,  0.12193052]],
      dtype=float32)

In [14]:
y = np.empty((10000,n_steps,10))
for step_ahead in range(1,10+1):
    y[:,:,step_ahead-1] = series[:,step_ahead:step_ahead+n_steps,0]
y_train = y[:7000]
y_valid= y[7000:9000]
y_test = y[9000:]

In [15]:
model =keras.models.Sequential([
    keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
    keras.layers.SimpleRNN(20,return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [16]:
def last_time_step_mse(y_true,y_pred):
    return keras.metrics.mean_squared_error(y_true[:,-1],y_pred[:,-1])

optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss='mse',optimizer=optimizer,metrics=[last_time_step_mse])

In [17]:
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self,units,activation='tanh',**kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units,activation=None)
        self.layer_norm = keras.layers.LayerNormalization()
        self.activation = keras.activations.get(activation)
    
    def call(self,inputs,states):
        outputs, new_states = self.simple_rnn_cell(inputs,states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [18]:
model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20),return_sequences=True,input_shape=[None,1]),
    keras.layers.RNN(LNSimpleRNNCell(20),return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [19]:
model = keras.models.Sequential([
    keras.layers.LSTM(20,return_sequences=True,input_shape=[None,1]),
    keras.layers.LSTM(20,return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [20]:
model = keras.models.Sequential([
    keras.layers.RNN(keras.layers.LSTMCell(20),return_sequences=True,input_shape=[None,1]),
    keras.layers.RNN(keras.layers.LSTMCell(20),return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(1))
])

In [21]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.txt',shakespeare_url)

with open(filepath) as f:
    shakespeare_txt = f.read()

In [22]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_txt)

In [23]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [24]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [25]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [26]:
max_id

39

In [27]:
dataset_size

1115394

In [28]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_txt]))-1

In [29]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [30]:
n_steps = 100
window_length = n_steps +1 
dataset = dataset.window(window_length,shift=1,drop_remainder=True)

In [31]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [32]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))

In [33]:
dataset = dataset.map(lambda x_batch, y_batch: (tf.one_hot(x_batch,depth=max_id),y_batch))

In [34]:
dataset = dataset.prefetch(1)

In [43]:
model = keras.models.Sequential([
    keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id],
                    dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128,return_sequences=True, dropout= 0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax'))
])

model.compile(loss='sparse_categorical_crossentropy',optimizer = 'adam')
history = model.fit(dataset,epochs=5)

Epoch 1/5
    513/Unknown - 168s 314ms/step - loss: 2.7067

KeyboardInterrupt: 

In [37]:
def preprocess(texts):
    x = np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(x,max_id)

In [39]:
x_new = preprocess(['How are you'])
y_pred = np.argmax(model(x_new),axis=-1)
tokenizer.sequences_to_texts(y_pred+1)[0][-1]

' '

In [41]:
def next_char(text,temperature=1):
    x_new = preprocess([text])
    y_proba = model(x_new)[0,-1:,:]
    rescaled_logits = tf.math.log(y_proba)/temperature
    char_id = tf.random.categorical(rescaled_logits,num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]


In [42]:
def complete_text(text,n_chars=50,temperature=1):
    for _ in range(n_chars):
        text += next_char(text,temperature)
    return text

In [ ]:
print(complete_text('t',temperature=0.2))
print(complete_text('w',temperature=1))
print(complete_text('w',temperature=2))

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift = n_steps, drop_remainder = True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = datset.batch(1)
dataset = dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))
dataset = dataset.map(lambda x_batch,y_batch: (tf.one_hot(x_batch,depth=max_id),y_batch))
dataset = datset.prefetch(1)

In [ ]:
#stateful RNN

model = keras.models.Sequential([
    keras.layers.GRU(128,return_sequences=True,stateful=True,
                    dropout=0.2, recurrent_dropout=0.2,
                    batch_input_shape=[batch_size,None,max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.2,recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax'))
])

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self,epoch,logs):
        self.model.reset_states()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')
model.fit(dataset,steps_per_epoch=steps_per_epoch,epochs=50,callbacks=[ResetStatesCallback()])

In [45]:
(x_train, y_train), (x_test,y_test) = keras.datasets.imdb.load_data()
np.array(x_train[0][:10],dtype='object')

array([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65], dtype=object)

In [48]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word,id_ in word_index.items()}

for id_, token in enumerate(('<pad>','<sos>','<unk>')):
    id_to_word[id_] = token
    
' '.join([id_to_word[id_] for id_ in x_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [52]:
pip install tensorflow_datasets


     ---------------------------------------- 4.7/4.7 MB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 52.3/52.3 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 140.1/140.1 kB 4.2 MB/s eta 0:00:00
     -------------------------------------- 110.5/110.5 kB 3.2 MB/s eta 0:00:00
     -------------------------------------- 211.7/211.7 kB 4.3 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21486 sha256=27c2df4f955269ae72f6b2f1e49e5ce71c3c3de22f20f28b36b4b7d95679e4ad
  Stored in directory: c:\users\ss263\appdata\local\pip\cache\wheels\54\aa\01\724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built promise


In [53]:
import tensorflow as tf
import tensorflow_datasets as tfds

datasets, info = tfds.load('imdb_reviews',as_supervised = True, with_info= True)
train_size = info.splits['train'].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ss263\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteE8WHE0\imdb_reviews-train…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ss263\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteE8WHE0\imdb_reviews-test.…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ss263\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteE8WHE0\imdb_reviews-unsup…

Dataset imdb_reviews downloaded and prepared to C:\Users\ss263\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [54]:
def preprocess(x_batch,y_batch):
    x_batch = tf.strings.substr(x_batch,0,300)
    x_batch = tf.strings.regex_replace(x_batch,b'<br\\s*/?>',b" ")
    x_batch = tf.strings.regex_replace(x_batch,b'[^a-zA-Z]',b" ")
    x_batch = tf.strings.split(x_batch)
    return x_batch.to_tensor(default_value=b'<pad>'), y_batch


In [56]:
from collections import Counter
vocabulary = Counter()

for x_batch, y_batch in datasets['train'].batch(32).map(preprocess):
    for review in x_batch:
        vocabulary.update(list(review.numpy()))

In [57]:
vocabulary.most_common()[:3]

[(b'<pad>', 224494), (b'the', 61156), (b'a', 38569)]

In [58]:
vocab_size =10000
truncated_vocabulary = [word for word,count in vocabulary.most_common()[:vocab_size]]

In [59]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary),dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words,word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init,num_oov_buckets)

In [64]:
table.lookup(tf.constant([b"This movie was faaaantastic".split()]))
                        

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   24,    12,    13, 10771]], dtype=int64)>

In [65]:
def encode_words(x_batch,y_batch):
    return table.lookup(x_batch), y_batch

train_set = datasets['train'].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [ ]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size+num_oov_buckets,embed_size,input_shape=[None]),
    keras.layers.GRU(128,return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history= model.fit(train_set,epochs=5)

In [1]:
K = keras.backend
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs,0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128,return_sequences=True)(z,mask=mask)
z = keras.layers.GRU(128)(z,mask=mask)
outputs = keras.layers.Dense(1,activation='sigmoid')(z)
model= keras.Model(inputs=[inputs],outputs=[outputs])

NameError: name 'keras' is not defined

In [ ]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None],dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None],dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[],dtype=np.int32)

embeddings= keras.layers.Embedding(vocab_size,embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h,state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer= keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell,sampler,output_layer=output_layer)
final_outputs, final_state,final_sequence_lengths = decoder(
decoder_embeddings, initial_state=encoder_state, sequence_length=sequence_lengths)

y_proba = tf.nn.softmax(final_outputs.rnn_output)
model = keras.Model(inputs=[encoder_inputs,decoder_inputs,sequence_lengths],outputs=[y_proba])

In [ ]:
keras.layers.Bidirectional(keras.layers.GRU(10,return_sequences=True))

In [ ]:
beam_width = 10

decoder = tfa.seq2seq.bean_search_decoder.BeamSearchDecoder(
cell = decoder_cell, beam_width = beam_width,output_layer = output_layer)
decoder_initial_state = tfa.seq2seq.beam_search_decoder.tile_batch(encoder_state, multiplier=beam_width)

outputs, _, _ = decoder(embedding_decoder,stat_tokens=stat_tokens, end_token=end_token,initial_state=decoder_initial_state)